# Table of contents
1. [Data Preparation](#dprep)
2. [Exploratory Data Analysis](#expda)
5. [Feature Engineering](#fe)
4. [Data Cleansing](#dclean)
5. [Modeling](#model)
6. [Evaluation](#eval)

## 1. Data Preparation <a name="dprep"></a>

In [6]:
# Import seluruh library yang diperlukan
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler 
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error as mse

In [3]:
# Data Preparation
def load_data(main_path, diag_path, proc_path):
    main = pd.read_csv(main_path)
    diag = pd.read_csv(diag_path)
    proc = pd.read_csv(proc_path)

    return main, diag, proc

In [ ]:
df_main, df_diag, df_proc = load_data('sampling_healtkathon2022/sampling_healtkathon2022.csv', 
                                    'sampling_healthkathon2022_diagnosa/sampling_healthkathon2022_diagnosa.csv', 
                                    'sampling_healthkathon2022_procedure/sampling_healthkathon2022_procedure.csv')

## 2. Exploratory Data Analysis <a name="expda"></a>

## 3. Feature Engineering <a name="fe"></a>

In [9]:
# Ngambil occuring

def merge_main_diag_proc(main, diag, proc):
    # Occur Diagnosa
    occur = pd.DataFrame()
    occur = occur.assign(occur_diagnosis = diag.groupby('id').size()) 
    gabungan_diag = main.merge(occur, on='id', how='left')

    # Occur Procedure
    occur = pd.DataFrame()
    occur = occur.assign(occur_procedure = proc.groupby('id').size()) 
    gabungan_final = gabungan_diag.merge(occur, on='id', how='left')
    
    return gabungan_final

In [ ]:
df_merged = merge_main_diag_proc(df_main,df_diag,df_proc)

In [10]:
df_merged

,id,id_peserta,dati2,typefaskes,usia,jenkel,pisat,tgldatang,tglpulang,jenispel,...,cbg,kelasrawat,kdsa,kdsp,kdsr,kdsi,kdsd,label,occur_diagnosis,occur_procedure
0,165666,486,17,KL,48,P,1.0,2018-07-25T17:00:00.000Z,2018-07-25T17:00:00.000Z,2,...,Q-5-42-0,3,None,None,None,None,None,0,1.0,NaN
1,1010828,520,17,A,63,L,1.0,2019-05-27T17:00:00.000Z,2019-05-30T17:00:00.000Z,1,...,D-4-13-III,1,None,None,None,None,None,0,2.0,1.0
2,166042,523,17,KL,53,P,1.0,2019-07-16T17:00:00.000Z,2019-07-16T17:00:00.000Z,2,...,Q-5-44-0,3,None,None,None,None,None,0,3.0,NaN
3,168937,549,17,KL,54,P,1.0,2019-10-17T17:00:00.000Z,2019-10-17T17:00:00.000Z,2,...,Q-5-44-0,3,None,None,None,None,None,0,3.0,NaN
4,1005899,549,17,A,53,P,1.0,2018-04-18T17:00:00.000Z,2018-04-18T17:00:00.000Z,2,...,Q-5-44-0,3,None,None,None,None,None,0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11401877,9983563,8527704,301,B,0,P,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,...,P-8-17-I,3,None,None,None,None,None,0,2.0,4.0
11401878,11053870,8527709,233,A,0,P,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,...,P-8-13-II,2,None,None,None,None,None,0,2.0,5.0
11401879,7461049,8527751,118,B,0,L,4.0,2020-12-30T17:00:00.000Z,2021-01-06T17:00:00.000Z,1,...,P-8-08-I,3,None,None,None,None,None,0,2.0,1.0
11401880,1075162,8527853,101,SC,0,L,5.0,2020-12-29T17:00:00.000Z,2020-12-31T17:00:00.000Z,1,...,P-8-08-I,3,None,None,None,None,None,0,1.0,1.0


In [11]:
def drop_columns (merged):
    merged = merged.drop(columns=['id'])
    merged = merged.drop(columns=['id_peserta'])
    merged['biaya_bagi100'] = merged['biaya']/100
    merged = merged.drop(columns=['biaya'])

    return merged

In [ ]:
merged_dropped = drop_columns(df_merged)

In [ ]:
merged_dropped

In [ ]:
# Menghitung selisih antar tanggal

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

def process_difference (merged_dropped):
    merged_dropped["Selisih"] = " "

    for i in range(len(merged_dropped)) :
        
        x = merged_dropped.iloc[i]['tgldatang']
        y = merged_dropped.iloc[i]['tglpulang']

        if x == y :
            merged_dropped.at[i,'Selisih'] = 0

        else :

            TanggalX = x[0:10]
            TanggalY = y[0:10]

            Selisih = days_between(TanggalX, TanggalY)
            merged_dropped.at[i, 'Selisih'] = Selisih

    merged_dropped = merged_dropped.drop(columns=['tgldatang'])
    merged_dropped = merged_dropped.drop(columns=['tglpulang'])

    return merged_dropped


In [ ]:
with_selisih = process_difference(merged_dropped)

## 4. Data Cleansing <a name="dclean"></a>

In [19]:
def process_na(no_na):
    no_na['jenkel'] = no_na['jenkel'].fillna(no_na['jenkel'].mode()[0])
    no_na['pisat'] = no_na['pisat'].fillna(no_na['pisat'].mode()[0])
    no_na['diagfktp'] = no_na['diagfktp'].fillna(no_na['diagfktp'].mode()[0])
    no_na['jenispulang'] = no_na['jenispulang'].fillna(no_na['jenispulang'].mode()[0])
    no_na['occur_procedure'] = no_na['occur_procedure'].fillna(0)
    no_na['occur_procedure'] = no_na['occur_procedure'].fillna(0)


    no_na['politujuan'] = no_na['politujuan'].fillna('ZZZ')
    no_na['kdsa'] = no_na['kdsa'].fillna('ZZZ')
    no_na['kdsp'] = no_na['kdsp'].fillna('ZZZZ')
    no_na['kdsr'] = no_na['kdsr'].fillna('ZZZZZ')
    no_na['kdsi'] = no_na['kdsi'].fillna('ZZZZZZ')
    no_na['kdsd'] = no_na['kdsd'].fillna('ZZZZZZZ')

    return no_na

,dati2,typefaskes,usia,jenkel,pisat,tgldatang,tglpulang,jenispel,politujuan,diagfktp,...,kdsa,kdsp,kdsr,kdsi,kdsd,label,occur_diagnosis,occur_procedure,biaya_bagi100,Selisih
0,17,KL,48,P,1.0,2018-07-25T17:00:00.000Z,2018-07-25T17:00:00.000Z,2,THT,L02.8,...,None,None,None,None,None,0,1.0,0.0,1843.0,0
1,17,A,63,L,1.0,2019-05-27T17:00:00.000Z,2019-05-30T17:00:00.000Z,1,ZZZ,R23.1,...,None,None,None,None,None,0,2.0,1.0,106284.0,3
2,17,KL,53,P,1.0,2019-07-16T17:00:00.000Z,2019-07-16T17:00:00.000Z,2,INT,E10.5,...,None,None,None,None,None,0,3.0,0.0,1873.0,0
3,17,KL,54,P,1.0,2019-10-17T17:00:00.000Z,2019-10-17T17:00:00.000Z,2,MAT,H54.2,...,None,None,None,None,None,0,3.0,0.0,1873.0,0
4,17,A,53,P,1.0,2018-04-18T17:00:00.000Z,2018-04-18T17:00:00.000Z,2,SAR,M54.5,...,None,None,None,None,None,0,1.0,0.0,3816.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11401877,301,B,0,P,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,ZZZ,Z71.8,...,None,None,None,None,None,0,2.0,4.0,36451.0,4
11401878,233,A,0,P,4.0,2020-12-30T17:00:00.000Z,2021-01-03T17:00:00.000Z,1,ZZZ,P59.9,...,None,None,None,None,None,0,2.0,5.0,122732.0,4
11401879,118,B,0,L,4.0,2020-12-30T17:00:00.000Z,2021-01-06T17:00:00.000Z,1,ZZZ,P24.8,...,None,None,None,None,None,0,2.0,1.0,40990.0,7
11401880,101,SC,0,L,5.0,2020-12-29T17:00:00.000Z,2020-12-31T17:00:00.000Z,1,ZZZ,P03,...,None,None,None,None,None,0,1.0,1.0,39924.0,2


In [ ]:
cleanril = process_na(with_selisih)

cleanril

In [27]:
cleanril.to_csv('clean_final.csv', index=False)

In [1]:
# ==================== CHECKPOINT ====================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler 
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error as mse

clean = pd.read_csv('clean_final.csv')

In [2]:
Pred = clean

In [3]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11401882 entries, 0 to 11401881
Data columns (total 21 columns):
 #   Column           Dtype  
---  ------           -----  
 0   dati2            int64  
 1   typefaskes       object 
 2   usia             int64  
 3   jenkel           object 
 4   pisat            float64
 5   jenispel         int64  
 6   politujuan       object 
 7   diagfktp         object 
 8   jenispulang      float64
 9   cbg              object 
 10  kelasrawat       int64  
 11  kdsa             object 
 12  kdsp             object 
 13  kdsr             object 
 14  kdsi             object 
 15  kdsd             object 
 16  label            int64  
 17  occur_diagnosis  float64
 18  occur_procedure  float64
 19  biaya_bagi100    float64
 20  Selisih          int64  
dtypes: float64(5), int64(6), object(10)
memory usage: 1.8+ GB


In [12]:
# Pembuatan data training
def convert_to_train(clean):
    satufull = clean.loc[clean['label'] == 1] # Pengambilan yang labelnya 1
    nolfull = clean.loc[clean['label'] == 0] # Pengambilan yang label 0

    satoversamp =  satufull.sample(frac =.7)

    satufull = satufull.append(satoversamp)

    jumsat = int(len(satufull))
    jumnol = int(len(nolfull)/3)
    satu = satufull.sample(n=(jumsat))
    nol = nolfull.sample(n=(jumnol)) # Pengambilan label 0 sejumlah banyaknya label 1

    Train = nol.append(satu)
    Train = Train.sample(frac = 1)

    X = Train.drop(columns=['label'])
    y = Train.label

    X_train_pre, X_test, y_train_pre, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    oversample = RandomOverSampler(sampling_strategy=0.08)

    X_train, y_train = oversample.fit_resample(X_train_pre, y_train_pre)

    return X_train, X_test, y_train, y_test

In [14]:
X_train, X_test, y_train, y_test = convert_to_train(clean)

C:\Users\faris\AppData\Local\Temp\ipykernel_17480\3609992161.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  satufull = satufull.append(satoversamp)
C:\Users\faris\AppData\Local\Temp\ipykernel_17480\3609992161.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Train = nol.append(satu)


## 5. Modeling <a name="model"></a>

In [15]:
def model_fitting(X_train, X_test, y_train, y_test):
    momoi = CatBoostClassifier(n_estimators=700,
                        loss_function='CrossEntropy',
                        learning_rate=0.4375,
                        depth=4, task_type='GPU',
                        random_state=1,
                        verbose=False)

    pool_train = Pool(X_train, y_train,
                    cat_features = ['typefaskes', 'jenkel', 'politujuan', 'diagfktp', 'cbg', 'kdsa', 'kdsp', 'kdsr', 'kdsi', 'kdsd'])
                    
    pool_test = Pool(X_test, cat_features = ['typefaskes', 'jenkel', 'politujuan', 'diagfktp', 'cbg', 'kdsa', 'kdsp', 'kdsr', 'kdsi', 'kdsd'])

    momoi.fit(pool_train)
    y_pred = momoi.predict(pool_test)
    cb_rmse = np.sqrt(mse(y_test, y_pred))
    print("RMSE:", np.mean(cb_rmse))

    return momoi

In [16]:
momoi = model_fitting(X_train, X_test, y_train, y_test)

RMSE: 0.18011936967414033


In [17]:
# Predict
Hasilpred = momoi.predict(Pred)

## 6. Evaluation <a name="eval"></a>

In [18]:
# Evaluation
def evaluate(clean, Hasilpred):
    y_true = clean.label
    y_pred = Hasilpred  # List of your dataframes


    def check(true,pred)  :
        tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
        Accuracy = (tn+tp) / (tn+fp+tp+fn)
        Precision = tp/(tp+fp)
        Recall = tp/(tp+fn)
        Specifity = tn/(tn+fp)  

        print("Accuracy    :", Accuracy, "\nPrecision   :", Precision, "\nRecall      :", Recall, "\nSpecifity   :", Specifity)

    check(y_true,y_pred)

    print("ROC AUC     :", roc_auc_score(y_true, y_pred))

In [19]:
evaluate(clean, Hasilpred)

Accuracy    : 0.9906735572250265 
Precision   : 0.6771372103780329 
Recall      : 0.6158366743366329 
Specifity   : 0.9959032433368344
ROC AUC     : 0.8058699588367337


In [11]:
# =================== START NEW CSV ===================

def pipeline_pred(main_path, diag_path, proc_path):
    df_main, df_diag, df_proc = load_data(main_path, diag_path, proc_path)
    df_merged = merge_main_diag_proc(df_main,df_diag,df_proc)
    merged_dropped = drop_columns(df_merged)
    with_selisih = process_difference(merged_dropped)
    pred = process_na(with_selisih)

    return pred

In [ ]:
pred_real = pipeline_pred('Pred/sampling2_healthkathon_2022_procedure.csv', 
                            'Pred/sampling2_healthkathon2022_diagnosa.csv', 
                            'Pred/sampling2_healthkathon2022_sep.csv')
pred_real

In [ ]:
hasil_pred = momoi.predict(pred_real)

In [ ]:
answer = pd.DataFrame()
answer['id']= clean['id']
answer['label'] = Hasilpred

print(answer)

# Save ke csv
answer.to_csv('answer.csv')